# Symmetry reduction

**Adapted from**: https://github.com/kalmarek/SymbolicWedderburn.jl/blob/tw/ex_sos/examples/ex_C4.jl

In [1]:
using Pkg
pkg"add https://github.com/kalmarek/SymbolicWedderburn.jl#tw/ex_sos"

import MutableArithmetics
const MA = MutableArithmetics
using MultivariatePolynomials
const MP = MultivariatePolynomials
using MultivariateBases
const MB = MultivariateBases

using DynamicPolynomials
@polyvar x[1:4]

    Cloning git-repo `https://github.com/kalmarek/SymbolicWedderburn.jl`
   Updating git-repo `https://github.com/kalmarek/SymbolicWedderburn.jl`
   Updating registry at `~/.julia/registries/General`
  Resolving package versions...
Updating `~/work/SumOfSquares.jl/SumOfSquares.jl/docs/Project.toml`
  [858aa9a9] + SymbolicWedderburn v0.1.0 `https://github.com/kalmarek/SymbolicWedderburn.jl#tw/ex_sos`
Updating `~/work/SumOfSquares.jl/SumOfSquares.jl/docs/Manifest.toml`
  [858aa9a9] + SymbolicWedderburn v0.1.0 `https://github.com/kalmarek/SymbolicWedderburn.jl#tw/ex_sos`


(DynamicPolynomials.PolyVar{true}[x₁, x₂, x₃, x₄],)

We would like to find the minimum value of the polynomial

In [2]:
poly = sum(x) + sum(x.^2)

x₁² + x₂² + x₃² + x₄² + x₁ + x₂ + x₃ + x₄

As we can decouple the problem for each `x[i]` for which `x[i] + x[i]^2` has
minimum value 0.25, we would expect to get `-1` as answer.
Can this decoupling be exploited by SumOfSquares as well ?
For this, we need to use a certificate that can exploit the permutation symmetry of the polynomial.
This is still a work in progress in SumOfSquares, so we need to define things here:

In [3]:
using SymbolicWedderburn
using PermutationGroups
using Cyclotomics
using SumOfSquares

function SymbolicWedderburn.OnPoints(basis::Union{MonomialVector, AbstractVector{<:Monomial}})
    basis_exps = Vector{Vector{Int}}(undef, length(basis))
    basis_dict = Dict{Vector{Int}, Int}()
    sizehint!(basis_dict, length(basis))

    for (i, b) in enumerate(basis)
        e = MP.exponents(b) # so that we allocate exponents only once
        basis_exps[i] = e
        basis_dict[e] = i
    end

    return SymbolicWedderburn.OnPoints(basis_exps, basis_dict)
end
function symmetry_adapted_basis(G, mvec)
    chars_vars = SymbolicWedderburn.characters_dixon(G)

    chars = chars_vars
    basis = mvec

    @assert all(χ.inv_of == first(chars).inv_of for χ in chars)

    induced_action = SymbolicWedderburn.OnPoints(basis)
    ccG_large = induced_action.(conjugacy_classes(first(chars)))

    let ccls = ccG_large, large_gens = induced_action.(gens(G)) # double check:
        G_large = PermGroup(large_gens)
        ccG_large = conjugacy_classes(G_large)
        @assert all(Set.(collect.(ccG_large)) .== Set.(collect.(ccls)))
    end

    chars_mvec = [SymbolicWedderburn.Character(values(χ), χ.inv_of, ccG_large) for χ in chars]

    vr_chars = SymbolicWedderburn.real_vchars(chars_mvec)
    U = filter!(x->all(!iszero, x), [SymbolicWedderburn.matrix_projection(χ) for χ in vr_chars])
    R = map(U) do c_u
        u = last(c_u)
        if all(isreal, u)
            image_coeffs, pivots = SymbolicWedderburn.row_echelon_form(float.(u))
            dim = length(pivots)
            image_coeffs[1:dim, :]
        else
            throw("Not Implemented")
        end
    end

    return map(R) do Ri
        FixedPolynomialBasis(Ri * mvec)
    end
end
function MP.polynomialtype(::Type{<:MB.AbstractPolynomialVectorBasis{PT}}, T::Type) where PT
    C = MP.coefficienttype(PT)
    U = MA.promote_operation(*, C, T)
    V = MA.promote_operation(+, U, U)
    return MP.polynomialtype(PT, V)
end
struct SymmetricIdeal{CT, GT} <: Certificate.AbstractIdealCertificate
    cone::CT
    group::GT
end
SumOfSquares.matrix_cone_type(::Type{<:SymmetricIdeal{CT}}) where {CT} = SumOfSquares.matrix_cone_type(CT)
Certificate.get(::Type{<:SymmetricIdeal}, ::SumOfSquares.Certificate.GramBasisType) = Vector{MB.FixedPolynomialBasis}
Certificate.zero_basis_type(::Type{<:SymmetricIdeal}) = MB.MonomialBasis
Certificate.zero_basis(::SymmetricIdeal) = MB.MonomialBasis
Certificate.get(::SymmetricIdeal, ::Certificate.ReducedPolynomial, poly, domain) = poly
function Certificate.get(cert::SymmetricIdeal, ::Certificate.GramBasis, poly)
    basis = Certificate.maxdegree_gram_basis(MB.MonomialBasis, MP.variables(poly), MP.maxdegree(poly))
    return symmetry_adapted_basis(cert.group, basis.monomials)
end
G = PermGroup([perm"(1,2,3,4)"])

Permutation group on 1 generators
⟨(1,2,3,4)⟩


We can use this certificate as follows:

In [4]:
import CSDP
solver = CSDP.Optimizer
model = Model(solver)
@variable(model, t)
@objective(model, Max, t)
con_ref = @constraint(model, sum(x) + sum(x.^2) - t in SOSCone(), ideal_certificate = SymmetricIdeal(SOSCone(), G))
optimize!(model)
value(t)

Iter:  7 Ap: 9.00e-01 Pobj:  0.0000000e+00 Ad: 1.00e+00 Dobj:  5.8619776e-14 
Success: SDP solved
Primal objective value: 0.0000000e+00 
Dual objective value: 0.0000000e+00 
Relative primal infeasibility: 1.75e-09 
Relative dual infeasibility: 2.83e-50 
Real Relative Gap: 0.00e+00 
XZ Relative Gap: 8.00e-50 
DIMACS error measures: 2.63e-09 0.00e+00 2.83e-50 0.00e+00 0.00e+00 8.00e-50
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 9.92e-01 Pobj: -4.6075840e+00 Ad: 9.50e-01 Dobj:  1.7648750e+01 
Iter:  2 Ap: 1.00e+00 Pobj: -2.3800098e+00 Ad: 9.16e-01 Dobj:  4.1804032e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -1.0644340e+00 Ad: 8.91e-01 Dobj:  2.9724937e-02 
Iter:  4 Ap: 1.00e+00 Pobj: -1.0071943e+00 Ad: 9.32e-01 Dobj: -9.0159234e-01 
Iter:  5 Ap: 1.00e+00 Pobj: -1.0005883e+00 Ad: 9.52e-01 Dobj: -9.9300460e-01 
Iter:  6 Ap: 1.00e+00 Pobj: -1.0000420e+00 Ad: 9.80e-01 Dobj: -9.9969898e-01 
Iter:  7 Ap: 1.00e+00 Pobj: -1.0000028e+00 Ad: 1.00e+00 

-1.0000000004504892

We indeed find `-1`, let's verify that symmetry was exploited:

In [5]:
gram_matrix(con_ref)

SparseGramMatrix{Float64,FixedPolynomialBasis{DynamicPolynomials.Polynomial{true,Float64},Array{DynamicPolynomials.Polynomial{true,Float64},1}},Float64,SymMatrix{Float64}}(GramMatrix{Float64,FixedPolynomialBasis{DynamicPolynomials.Polynomial{true,Float64},Array{DynamicPolynomials.Polynomial{true,Float64},1}},Float64,SymMatrix{Float64}}[GramMatrix{Float64,FixedPolynomialBasis{DynamicPolynomials.Polynomial{true,Float64},Array{DynamicPolynomials.Polynomial{true,Float64},1}},Float64,SymMatrix{Float64}}([0.2500000000000006], FixedPolynomialBasis{DynamicPolynomials.Polynomial{true,Float64},Array{DynamicPolynomials.Polynomial{true,Float64},1}}(DynamicPolynomials.Polynomial{true,Float64}[x₁ - x₂ + x₃ - x₄])), GramMatrix{Float64,FixedPolynomialBasis{DynamicPolynomials.Polynomial{true,Float64},Array{DynamicPolynomials.Polynomial{true,Float64},1}},Float64,SymMatrix{Float64}}([0.25000000000000194 0.499999999999997; 0.499999999999997 1.000000000450487], FixedPolynomialBasis{DynamicPolynomials.Polyn

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*